# Preprocess Data

## Load data

In [ ]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

SUBSCRIPTION_ID = "757c4165-0823-49f7-9678-5a85fe5e17cc"
RESOURCE_GROUP_NAME = "e2e_aml_ref_imp_ds_rg"
WORKSPACE_NAME = "entml-dev-machinelearning001"
DATA_ASSET_NAME = "diabetes_mltable"
DATA_ASSET_VERSION = "4"


ml_client = MLClient(
    credential=DefaultAzureCredential(),
    subscription_id=SUBSCRIPTION_ID,
    resource_group_name=RESOURCE_GROUP_NAME,
    workspace_name=WORKSPACE_NAME,
)
data_asset = ml_client.data.get(
    name=DATA_ASSET_NAME,
    version=DATA_ASSET_VERSION,
)

In [ ]:
import mltable


tbl = mltable.load(data_asset.path)
df = tbl.to_pandas_dataframe()
df.head()

## Train, test split

In [ ]:
from sklearn.model_selection import train_test_split

RANDOM_STATE = 0
TEST_SIZE = 0.2


train, test = train_test_split(
    df, test_size=TEST_SIZE, random_state=RANDOM_STATE, shuffle=True
)

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler


scaler = StandardScaler(copy=True, with_mean=True, with_std=True)
scaler_model = scaler.fit(train)
train_norm_df = pd.DataFrame(data=scaler_model.transform(train), columns=df.columns)
test_norm_df = pd.DataFrame(data=scaler_model.transform(test), columns=df.columns)

In [ ]:
train_norm_df.head()

In [ ]:
test_norm_df.head()

In [ ]:
mean = {f"{scaler_model.feature_names_in_[i]}_mean": scaler_model.mean_[i] for i in range(len(scaler_model.feature_names_in_))}
scale = {f"{scaler_model.feature_names_in_[i]}_std": scaler_model.scale_[i] for i in range(len(scaler_model.feature_names_in_))}